In [1]:
import os
import sys
directory_path = os.path.abspath('/notebooks/')
if directory_path not in sys.path:
    sys.path.append(directory_path)

import pandas as pd
import numpy as np

In [2]:
policies_raw = pd.read_csv('../data/policies.csv', index_col=0)
cols_to_drop = ['zip', 'county_name', 'Agent_cd']
policies_raw = policies_raw.drop(cols_to_drop, axis=1)

Subset df according to 'split' variable

In [12]:
# subset df according to 'split' variable
train_policies_raw = policies_raw[policies_raw['split'] == 'Train']
test_policies_raw = policies_raw[policies_raw['split'] == 'Test']

2


In [ ]:
class DataValidation:
    def __init__(self, df):
        self.raw_data = df
        self.complete_data = None  # implement this later
        self.raw_catvars = self.raw_data.select_dtypes(exclude=[np.number])
        self.raw_numvars = self.raw_data.select_dtypes(include=[np.number])

    # returns numpy array of categorical variables as they appear in dataset
    def get_categoric(self):
        return self.raw_catvars.columns.values

    # returns numpy array of numerical variables as they appear in dataset
    def get_numeric(self):
        return self.raw_numvars.columns.values

    # returns df of pct missing data points for each predictor
    def get_missing(self):
        dict_missing = {'col':[], 'pct_missing':[]}

        for col in self.raw_data.columns:
            mean_missing = np.mean(self.raw_data[col].isnull())
            pct_missing = round(mean_missing * 100, 5)
            
            dict_missing['col'].append(col)
            dict_missing['pct_missing'].append(pct_missing)

        df_missing = pd.DataFrame(data=dict_missing)
        return df_missing

In [ ]:
policy_data = DataValidation(policies_raw)
# print(policy_data.get_categoric())
# print(policy_data.get_numeric())
# print(policy_data.get_missing())

Verify that there are no duplicate poicy ids before making set of dropped ids

In [ ]:
unique_id_counts = policy_data.raw_data['policy_id'].value_counts()
duplicates = [item for item in unique_id_counts if item != 1]
assert(len(duplicates) == 0) # there are no duplicate ids

In [ ]:
dropped_policy_ids = set()

# make copy of raw policy data df, then drop missing values
df_missing_vals = policy_data.raw_data
df_missing_vals = df_missing_vals.dropna(how='all').dropna(how='all', axis=1)

# store T/F values of isnull() for each policy id in a pd series
is_missing = df_missing_vals.isnull().any(axis=1)
for i in range(1, len(is_missing) + 1):
    if is_missing[i]:
        dropped_policy_ids.add(policy_data.raw_data['policy_id'][i])

Verify that len of dropped policy ids set is equal to total number of rows containing missing values

In [ ]:
is_missing_counts = np.unique(is_missing, return_counts=True)
assert(len(dropped_policy_ids) == is_missing_counts[1][1]) # flagged policy ids were dropped successfully

In [37]:
# policy_43210
policy_data.raw_data[policy_data.raw_data['policy_id'] == 'policy_43210']

,Quote_dt,discount,Home_policy_ind,state_id,quoted_amt,Prior_carrier_grp,credit_score,Cov_package_type,CAT_zone,policy_id,number_drivers,num_loaned_veh,num_owned_veh,num_leased_veh,total_number_veh,convert_ind,split,primary_parking
34089,2015-10-22,Yes,Y,NJ,"$2,238",Carrier_5,678.0,Low,1.0,policy_43210,5,1,1,1,3,NaN,Test,home/driveway
